## Imports

In [1]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob

## Read-in

In [2]:
# Get today's date
today = datetime.now()

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Jul. 22nd Jul. 29th


In [3]:
filename = glob.glob('redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [8]:
# # Filter dates between. If only using data looking back one week, this shouldn't change the df
# df = df[(df['SOLD DATE'] >= '2024-06-10') & (df['SOLD DATE'] <= '2024-06-17')]

In [7]:
df.sort_values(ascending=False, by='SOLD DATE')

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,2024-07-29,Condo/Co-op,2003 SE 27th Dr Unit 102-A,Homestead,FL,33035.0,220000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Homestead/2003-SE-27...,MARMLS,A11600809,N,Y,25.452080,-80.438109
40,PAST SALE,2024-07-29,Condo/Co-op,10009 W Okeechobee Rd #103,Hialeah,FL,33016.0,315000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hialeah/10009-W-Okee...,Beaches MLS,F10443009,N,Y,25.864624,-80.334997
102,PAST SALE,2024-07-29,Condo/Co-op,8390 SW 72nd Ave Unit 901 PH,Miami,FL,33143.0,585000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/8390-SW-72nd-A...,MARMLS,A11551420,N,Y,25.693107,-80.309864
92,PAST SALE,2024-07-29,Condo/Co-op,1900 Sunset Harbour Dr #2407,Miami Beach,FL,33139.0,1150000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1900-Sun...,MARMLS,A11541894,N,Y,25.795066,-80.144638
26,PAST SALE,2024-07-29,Condo/Co-op,1000 S Pointe Dr #1404,Miami Beach,FL,33139.0,3700000.0,3.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1000-S-P...,MARMLS,A11604509,N,Y,25.768527,-80.137591
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,PAST SALE,2024-07-22,Condo/Co-op,1451 Brickell Ave #2502,Miami,FL,33131.0,950000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1451-Brickell-...,MARMLS,A11540952,N,Y,25.758141,-80.192431
48,PAST SALE,2024-07-22,Condo/Co-op,401 69th St #1111,Miami Beach,FL,33141.0,320000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/401-69th...,MARMLS,A11564228,N,Y,25.854691,-80.122750
72,PAST SALE,2024-07-22,Condo/Co-op,2415 NW 16th St Rd #316,Miami,FL,33125.0,270000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2415-NW-16th-S...,MARMLS,A11585888,N,Y,25.789892,-80.236447
104,PAST SALE,2024-07-22,Condo/Co-op,1465 NE 123rd St #708,North Miami,FL,33161.0,177000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami/1465-NE-...,MARMLS,A11516621,N,Y,25.890867,-80.169031


In [9]:
df = df.sort_values(by='PRICE',ascending=False)
df_top_ten = df.head(10)

## Current Week's Values

In [10]:
# Number of condos sold
current_week_condo_sales = len(df)
print(len(df))

131


In [11]:
# Average sale price
average_sale_price = round(df['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$668,297


In [12]:
# Average PSF
average_price_psf = round(df['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$509


In [13]:
# Total sales dollar volume
current_week_condo_sales_total = df['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$87,546,915


In [14]:
# # Define the date range
# start_date = '2024-05-20'
# end_date = '2024-05-26'

# # Filter rows where 'SOLD DATE' is within the specified range
# second_week_df = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

## Previous Week's Values

In [15]:
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

Input Previous Week Condo Sales Total (number units sold):
140
Input Previous Week Condo Average Sales Price:
938812
Input Previous Week Condo Average PSF:
586
Input Previous Week Condo Sales total (ex: 198_000_000)
131000000


## Clean Data

In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [16]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [17]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [18]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'RANK', 'COLOR'],
      dtype='object')

In [19]:
pd.set_option('display.max_columns',None)

In [20]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,RANK,COLOR
60,PAST SALE,2024-07-22,Condo/Co-op,5263 Fisher Island Dr #5263,Miami Beach,FL,33109.0,10500000.0,4.0,4.5,BAYVIEW FISHER ISL CONDO,3790.0,NaN,1992.0,NaN,2770.0,9245.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/5263-Fis...,MARMLS,A11590009,N,Y,25.762689,-80.148415,1,orange


In [21]:
def popup_html(row):
    building_name = row['LOCATION']
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(building_name) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [22]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} - {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [23]:
m.save('index.html')

## Data snagger

In [24]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [25]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,RANK,COLOR
60,PAST SALE,2024-07-22,Condo/Co-op,5263 Fisher Island Dr #5263,Miami Beach,FL,33109.0,10500000.0,4.0,4.5,BAYVIEW FISHER ISL CONDO,3790.0,NaN,1992.0,NaN,2770.0,9245.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/5263-Fis...,MARMLS,A11590009,N,Y,25.762689,-80.148415,1,orange


In [ ]:
# , {df_top_ten[df_top_ten['PRICE'].idxmax()]['ADDRESS']} | Price ${df_top_ten[df_top_ten['PRICE'].idxmax()]['PRICE']:,.0f}

In [26]:
df_top_ten['PRICE'].iloc[0]

10500000.0

In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [27]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/condo_sales_week_ending_07292024


# CREATE TEMPLATE 

In [28]:
muni_set = set(df_top_ten['CITY'])

In [29]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [30]:
df_top_ten.reset_index(inplace=True,drop=True)

In [31]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [32]:
top_sale

'BAYVIEW FISHER ISL CONDO at 5263 Fisher Island Dr #5263 in Miami Beach'

In [33]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [34]:
# df_top_ten

## RESUME HERE

In [34]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops Miami-Dade weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f}M to ${df_top_ten.at[0,'PRICE']:,.0f}M
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m Miami-Dade County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m Miami-Dade County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Adam Farence
\033[1mRESEARCH:\033[0m 
\033[1mSocial:\033[0m #MiamiDade condo sales XXXXX
\033[1mART:\033[0m

Interactive map

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, Miami-Dade County, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

Miami-Dade County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} sales average from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: BAYVIEW FISHER ISL CONDO closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $1,685,000M to $10,500,000M
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #MiamiDade condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade County, weekly condo sales, Key Biscayne, Miami Beach, Aventura, Miami, Bal Harbour, Sunny Isles Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 131 condo sales totaling $87,546,915 million from Jul. 22nd to Jul. 29th. The previous week, brokers closed 140 condo sales totaling $131,000,000.

Last week’s units sold for an average of $668,297, lower than the $938,812 sales average from the pr

In [35]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [36]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [37]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf")

print(story_checklist)


HED: BAYVIEW FISHER ISL CONDO closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $1,685,000M to $10,500,000M
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #MiamiDade condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade County, weekly condo sales, Key Biscayne, Miami Beach, Aventura, Miami, Bal Harbour, Sunny Isles Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 131 condo sales totaling $87,546,915 million from Jul. 22nd to Jul. 29th. The previous week, brokers closed 140 condo sales totaling $131,000,000.

Last week’s units sold for an average of $668,297, lower than the $938,812 sales average from the pr

In [45]:
print(df_top_ten['URL'].iloc[0])

https://www.redfin.com/FL/Miami-Beach/5263-Fisher-Island-Dr-33109/unit-5263/home/43430622


In [46]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Miami-Beach/1000-S-Pointe-Dr-33139/unit-1404/home/43267663


In [47]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Key-Biscayne/201-Crandon-Blvd-33149/unit-1024/home/42918525


In [48]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Miami-Beach/400-S-Pointe-Dr-33139/unit-2010/home/42801754


In [53]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Miami-Beach/5263-Fisher-Island-Dr-33109/unit-5263/home/43430622


In [54]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Sunny-Isles-Beach/330-Sunny-Isles-Blvd-33160/unit-5-2508/home/56744643


In [44]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-07-22,Condo/Co-op,5263 Fisher Island Dr #5263,Miami Beach,FL,33109.0,10500000.0,4.0,4.5,BAYVIEW FISHER ISL CONDO,3790.0,NaN,1992.0,NaN,2770.0,9245.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/5263-Fis...,MARMLS,A11590009,N,Y,25.762689,-80.148415,1,orange,BAYVIEW FISHER ISL CONDO at 5263 Fisher Island...
1,PAST SALE,2024-07-29,Condo/Co-op,1000 S Pointe Dr #1404,Miami Beach,FL,33139.0,3700000.0,3.0,3.0,MURANO AT PORTOFINO CONDO,2190.0,NaN,2002.0,NaN,1689.0,3024.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1000-S-P...,MARMLS,A11604509,N,Y,25.768527,-80.137591,2,blue,MURANO AT PORTOFINO CONDO at 1000 S Pointe Dr ...
2,PAST SALE,2024-07-25,Condo/Co-op,10295 Collins Ave #1505,Bal Harbour,FL,33154.0,3650000.0,2.0,3.0,RESIDENCES AT ONE BAL HARB,2221.0,NaN,2007.0,NaN,1643.0,3649.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10295-Co...,MARMLS,A11518907,N,Y,25.898952,-80.123405,3,blue,RESIDENCES AT ONE BAL HARB at 10295 Collins Av...
3,PAST SALE,2024-07-26,Condo/Co-op,20155 NE 38th Ct Unit TS-03,Aventura,FL,33180.0,3550000.0,3.0,4.5,TOWERS OF PORTO VITA-NORT,3385.0,NaN,2003.0,NaN,1049.0,4530.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/20155-NE-38...,MARMLS,A11499190,N,Y,25.962401,-80.126022,4,blue,TOWERS OF PORTO VITA-NORT at 20155 NE 38th Ct ...
4,PAST SALE,2024-07-22,Condo/Co-op,480 NE 31 Ph 5201,Miami,FL,33137.0,3350000.0,4.0,4.5,GRAN PARAISO CONDO,2650.0,NaN,2019.0,NaN,1264.0,3073.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/480-NE-31st-St...,MARMLS,A11578451,N,Y,25.806186,-80.187176,5,blue,GRAN PARAISO CONDO at 480 NE 31 Ph 5201 in Miami
5,PAST SALE,2024-07-26,Condo/Co-op,900 Biscayne Blvd Ph 6301,Miami,FL,33132.0,3180000.0,4.0,4.5,900 BISCAYNE BAY CONDO,3199.0,NaN,2008.0,NaN,994.0,2915.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/900-Biscayne-B...,MARMLS,A11510295,N,Y,25.783555,-80.189982,6,blue,900 BISCAYNE BAY CONDO at 900 Biscayne Blvd Ph...
6,PAST SALE,2024-07-22,Condo/Co-op,330 Sunny Isles Blvd Unit 5-2508,Sunny Isles Beach,FL,33160.0,1825000.0,3.0,3.5,PARQUE TOWERS CONDO,1892.0,NaN,2019.0,NaN,965.0,1578.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/33...,MARMLS,A11574520,N,Y,25.929401,-80.126023,7,blue,PARQUE TOWERS CONDO at 330 Sunny Isles Blvd Un...
7,PAST SALE,2024-07-26,Condo/Co-op,2127 Brickell Ave #1204,Miami,FL,33129.0,1765000.0,2.0,2.0,BRISTOL TOWER CONDO,1610.0,NaN,1993.0,NaN,1096.0,2728.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2127-Brickell-...,MARMLS,A11496680,N,Y,25.752160,-80.199342,8,blue,BRISTOL TOWER CONDO at 2127 Brickell Ave #1204...
8,PAST SALE,2024-07-26,Condo/Co-op,400 S Pointe Dr #2010,Miami Beach,FL,33139.0,1750000.0,2.0,2.0,SOUTH POINTE TOWERS CONDO,1559.0,NaN,1987.0,NaN,1123.0,2301.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/400-S-Po...,MARMLS,A11563370,N,Y,25.767539,-80.134220,9,blue,SOUTH POINTE TOWERS CONDO at 400 S Pointe Dr #...
9,PAST SALE,2024-07-23,Condo/Co-op,201 Crandon Blvd #1024,Key Biscayne,FL,33149.0,1685000.0,2.0,2.0,KEY COLONY NO 1,1531.0,NaN,1978.0,NaN,1101.0,1623.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/201-Cra...,MARMLS,A11470485,N,Y,25.696814,-80.158094,10,blue,KEY COLONY NO 1 at 201 Crandon Blvd #1024 in K...


## Time on Market Calculator

In [52]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 6, 10) ## List (Earlier) date
date2 = datetime(2024, 7, 29) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

49
